In [29]:
import pandas as pd
import time
from datetime import datetime
import math
import ccxt
import json

In [30]:
with open('binance.json') as f:
    binance_api = json.load(f)

In [31]:
exchange_id = 'binance'
exchange_class = getattr(ccxt, exchange_id)

exchange = exchange_class({
    "apiKey": binance_api['apiKey'],
    "secret": binance_api['secret'],
})

In [32]:
markets = exchange.fetchMarkets()
market_symbols = [market['symbol'] for market in markets if market['type']=='spot']

In [33]:
def create_chain(orders_type, start, middle, finish,  start_price, middle_price, finish_price, res):   
    return {
        'type': orders_type,
        'start': start['symbol'], # USDT                 
        'middle': middle['symbol'],      # BTC                 
        'finish': finish['symbol'],      # ETH 
        'start_price': start_price,     
        'middle_price': middle_price,       
        'finish_price': finish_price,
        'Result' : res
    }
    

def get_chains(market_symbols, quotes, base_currency='USDT'):
    chains = []
    AMMOUNT=100.0

    for symbol in market_symbols:
        try:
            token1 = symbol.split('/')[0] 
            token2 = symbol.split('/')[1]
            #ищем промежуточное звено треугольника и проверяем результат в 2 стороны
            if (token1!=base_currency and token2!=base_currency):
                start = quotes[f'{token1}/{base_currency}']
                middle = quotes[f'{token1}/{token2}']
                finish = quotes[f'{token2}/{base_currency}']
                if  start['ask']!=0 and start['bid']!=0 and middle['ask']!=0 and middle['bid']!=0 and finish['ask']!=0 and finish['bid']!=0:
                    #USDT-ETH-BTC
                    chains.append(
                                create_chain(
                                    'BUY_SELL_SELL',
                                    start,         #ETH/USDT
                                    middle,        #ETH/BTC
                                    finish,        #BTC/USDT
                                    start['ask'],  #ETH/USDT ASK
                                    middle['bid'], #ETH/BTC BID
                                    finish['bid'], #BTC/USDT BID
                                    float(AMMOUNT/start['ask']*middle['bid']*finish['bid'] - AMMOUNT)
                                )
                    )
                    #USDT-BTC-ETH
                    chains.append(
                                create_chain(
                                    'BUY_BUY_SELL',
                                    finish,       #BTC/USDT
                                    middle,       #ETH/BTC
                                    start,        #ETH/USDT
                                    finish['ask'], #BTC/USDT ASK
                                    middle['ask'], ##ETH/BTC ASK
                                    start['bid'],  #ETH/USDT BID
                                    float(AMMOUNT/finish['ask']/middle['ask']*start['bid'] - AMMOUNT)
                                )
                   )

        except KeyError:
            pass
    return pd.DataFrame(chains)


In [48]:
def place_buy_order(symbol, amount, price, delta):
    order_price = price*(1+delta) 
    order = exchange.create_limit_buy_order(symbol, amount, order_price)
    print(order)
    return order

def place_sell_order(symbol, amount, price, delta):
    order_price = price*(1-delta) 
    order = exchange.create_limit_sell_order(symbol, amount, order_price)
    print(order)
    return order 

def place_trade_orders(chain, initial_amount):
    final_amount = 0.0
    delta = 0.001 # увеличение/уменьшение цены лимитного ордера для возможного проскальзывания
    orders_type = chain['type']
    pair_1 = chain['start']
    pair_2 = chain['middle']
    pair_3 = chain['finish']
    price_1 = chain['start_price']
    price_2 = chain['middle_price']
    price_3 = chain['finish_price']
                
    if orders_type == 'BUY_BUY_SELL':
        quantity_1 = initial_amount/price_1
        order_1 = place_buy_order(pair_1, quantity_1, price_1, delta)
        
        quantity_2 = quantity_1/price_2
        order_2 = place_buy_order(pair_2, quantity_2, price_2, delta)
        
        quantity_3 = quantity_2
        order_3 = place_sell_order(pair_3, quantity_3, price_3, delta)
        
    elif orders_type == 'BUY_SELL_SELL':
        quantity_1 = initial_amount/price_1
        order_1 = place_buy_order(pair_1, quantity_1, price_1, delta)
        
        quantity_2 = quantity_1
        order_2 = place_sell_order(pair_2, quantity_2, price_2, delta)
        
        quantity_3 = quantity_2 * price_2
        order_3 = place_sell_order(pair_3, quantity_3, price_3, delta)
        #todo сделать проверку ордеров. По какой цене исполнились и исполнились ли вообще. 
        #Если все выполнено то считать результат
    ##############
    time.sleep(60) #ждем минуту чтобы исключить возможный флипинг котировки    
    ##############
    return final_amount

def filter_profitable_chains(df):
    MIN_PROFIT = 0.5
    AMMOUNT = 25.0
    profit = df.loc[df['Result']>MIN_PROFIT]
    if not profit.empty:
        print(f"Time-{datetime.now().strftime('%H:%M:%S')}")
        print(profit)
        place_trade_orders(profit.iloc[0], AMMOUNT)


In [ ]:
while(1):    
    quotes = exchange.fetchBidsAsks()
    chains = get_chains(market_symbols, quotes, 'USDT')
    sorted_chains = chains.sort_values(by='Result', ascending=False)
    filter_profitable_chains(sorted_chains)
    time.sleep(1)

In [ ]:
#throw_exactly_matched_exception(self.exceptions['exact'], error, feedback)
#InvalidOrder: binance {"code":-1013,"msg":"Filter failure: MIN_NOTIONAL"}
#price * quantity is too low to be a valid order for the symbol.

# Donation(BNB, ETH) --> 0xD1fd90B9c2585354923c33e90878c26cA83Fe0F9